In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn import preprocessing 
from sklearn.metrics import accuracy_score,mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
from catboost import CatBoostClassifier 

In [34]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sample = pd.read_csv('SampleSubmission.csv')

In [4]:
train

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0000,0.000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0000,0.000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0000,6.282,NaN,956940.0,NaN,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0000,0.000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0000,0.000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55995,Apcnt_999968,3740.0,0.01730,0.0000,0.0000,0.000,770998.0,9637475.0,4047934.0,11641992.0,...,0.000000,9.09,0.730040,NaN,0.0,lending,591.048913,0.000000,0.241598,no
55996,Apcnt_999972,3360.0,2.01145,0.6252,0.0000,0.000,NaN,927765.0,NaN,NaN,...,0.297913,30.30,0.498208,NaN,0.0,lending,NaN,0.000000,NaN,yes
55997,Apcnt_999980,3500.0,0.76640,0.0000,0.0000,0.000,118645.0,3662435.0,3662435.0,3585024.0,...,0.550000,3.03,0.450512,0.0,0.0,lending,1252.474467,2.000000,0.163153,no
55998,Apcnt_999988,3280.0,0.05235,2.0916,2.2212,0.000,NaN,3458599.0,NaN,115533.0,...,0.942854,0.00,0.705048,1.0,0.0,charge,NaN,0.000000,0.002997,no


In [5]:
test.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,...,NaN,0.825000,1.01,0.800000,NaN,0.0,charge,NaN,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,...,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,NaN,0.504974,0.043525
2,Apcnt_1000052,NaN,0.27505,0.0600,0.0000,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.00,NaN,NaN,NaN,charge,NaN,0.000000,NaN
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,...,NaN,0.916663,2.02,0.464224,NaN,NaN,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,...,NaN,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124


In [6]:
train.shape

(56000, 52)

In [7]:
test.shape

(24000, 51)

In [8]:
train.isnull().sum()

Applicant_ID          0
form_field1        2529
form_field2        3844
form_field3         355
form_field4         355
form_field5         355
form_field6       13360
form_field7        5163
form_field8       13360
form_field9        8008
form_field10        355
form_field11      31421
form_field12       9895
form_field13       5889
form_field14          0
form_field15      22475
form_field16      13036
form_field17      11151
form_field18      10402
form_field19          4
form_field20        355
form_field21      15854
form_field22      20400
form_field23      28123
form_field24      13297
form_field25       5450
form_field26       7438
form_field27       9299
form_field28        355
form_field29        355
form_field30      25509
form_field31      39408
form_field32       5450
form_field33       1256
form_field34        355
form_field35      23148
form_field36       1995
form_field37       5450
form_field38        355
form_field39       4211
form_field40      43729
form_field41    

In [9]:
columns = train.columns

In [10]:
for i in columns:
    train[i].fillna(-999,inplace=True)

In [11]:
train.isnull().sum()

Applicant_ID      0
form_field1       0
form_field2       0
form_field3       0
form_field4       0
form_field5       0
form_field6       0
form_field7       0
form_field8       0
form_field9       0
form_field10      0
form_field11      0
form_field12      0
form_field13      0
form_field14      0
form_field15      0
form_field16      0
form_field17      0
form_field18      0
form_field19      0
form_field20      0
form_field21      0
form_field22      0
form_field23      0
form_field24      0
form_field25      0
form_field26      0
form_field27      0
form_field28      0
form_field29      0
form_field30      0
form_field31      0
form_field32      0
form_field33      0
form_field34      0
form_field35      0
form_field36      0
form_field37      0
form_field38      0
form_field39      0
form_field40      0
form_field41      0
form_field42      0
form_field43      0
form_field44      0
form_field45      0
form_field46      0
form_field47      0
form_field48      0
form_field49      0


In [12]:
test.isnull().sum()

Applicant_ID        0
form_field1      1110
form_field2      1709
form_field3       146
form_field4       146
form_field5       146
form_field6      5604
form_field7      2231
form_field8      5604
form_field9      3400
form_field10      147
form_field11    13398
form_field12     4183
form_field13     2463
form_field14        0
form_field15     9592
form_field16     5474
form_field17     4695
form_field18     4369
form_field19        0
form_field20      147
form_field21     6707
form_field22     8724
form_field23    12125
form_field24     5605
form_field25     2256
form_field26     3172
form_field27     3910
form_field28      147
form_field29      147
form_field30    10908
form_field31    16810
form_field32     2256
form_field33      495
form_field34      147
form_field35     9866
form_field36      903
form_field37     2256
form_field38      147
form_field39     1829
form_field40    18828
form_field41    16349
form_field42      578
form_field43      250
form_field44     2362
form_field

In [13]:
column = test.columns

In [14]:
for i in column:
    test[i].fillna(-999,inplace=True)

In [15]:
test.isnull().sum()

Applicant_ID    0
form_field1     0
form_field2     0
form_field3     0
form_field4     0
form_field5     0
form_field6     0
form_field7     0
form_field8     0
form_field9     0
form_field10    0
form_field11    0
form_field12    0
form_field13    0
form_field14    0
form_field15    0
form_field16    0
form_field17    0
form_field18    0
form_field19    0
form_field20    0
form_field21    0
form_field22    0
form_field23    0
form_field24    0
form_field25    0
form_field26    0
form_field27    0
form_field28    0
form_field29    0
form_field30    0
form_field31    0
form_field32    0
form_field33    0
form_field34    0
form_field35    0
form_field36    0
form_field37    0
form_field38    0
form_field39    0
form_field40    0
form_field41    0
form_field42    0
form_field43    0
form_field44    0
form_field45    0
form_field46    0
form_field47    0
form_field48    0
form_field49    0
form_field50    0
dtype: int64

In [16]:
for i in train.columns:
    print(train[i].dtype)

object
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
int64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
object
float64
float64
float64
object


In [17]:
def replace_form_field(val):
    if val =='charge':
        return 0
    else:
        return 1
train['form_field47']= train['form_field47'].apply(replace_form_field, 1)

In [18]:
def replace_form_field(val):
    if val =='charge':
        return 0
    else:
        return 1
train['form_field47']= test['form_field47'].apply(replace_form_field, 1)

In [19]:
def replace_default_status(val):
    if val =='yes':
        return 0
    else:
        return 1
train['default_status']= train['default_status'].apply(replace_default_status, 1)

In [20]:
train.describe()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
count,56000.000000,56000.000000,56000.000000,56000.000000,56000.000000,5.600000e+04,5.600000e+04,5.600000e+04,5.600000e+04,5.600000e+04,...,56000.000000,56000.000000,56000.000000,56000.000000,56000.000000,24000.000000,5.600000e+04,56000.000000,5.600000e+04,56000.000000
mean,3288.988125,-68.061282,-5.287392,-5.486368,-4.389031,4.752342e+05,6.232170e+06,1.999799e+06,1.127799e+07,1.178069e+07,...,-23.241859,-3.565496,-95.519653,-558.640946,-283.647714,0.355542,1.910105e+05,-5.290536,4.818161e+05,0.755089
std,950.560828,252.735737,79.400587,79.418227,80.131613,1.278889e+06,1.833216e+07,3.605357e+06,1.888158e+07,2.662648e+07,...,151.783727,100.964839,294.636580,496.023149,450.532984,0.478687,1.313103e+06,79.391884,5.239461e+06,0.430038
min,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,...,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.000000,-9.990000e+02,-999.000000,-9.990000e+02,0.000000
25%,3336.000000,0.047700,0.000000,0.000000,0.000000,0.000000e+00,3.248150e+05,0.000000e+00,4.217732e+05,4.170080e+05,...,0.000000,2.020000,0.349148,-999.000000,-999.000000,0.000000,-9.990000e+02,0.000000,7.502524e-03,1.000000
50%,3474.000000,0.226700,0.060000,0.000000,0.000000,4.006700e+04,2.174704e+06,3.765520e+05,3.712227e+06,3.641234e+06,...,0.206250,5.050000,0.548080,-999.000000,0.000000,0.000000,5.141139e+01,0.000000,9.600000e-02,1.000000
75%,3610.000000,0.675700,1.270800,0.000000,0.000000,3.023502e+05,6.335740e+06,2.361230e+06,1.403308e+07,1.315909e+07,...,0.611116,10.100000,0.739474,0.000000,0.000000,1.000000,5.096936e+02,1.158480,3.800000e-01,1.000000
max,3900.000000,18.015050,57.371600,91.672200,407.748600,5.313546e+07,2.158794e+09,1.037397e+08,3.200533e+08,2.191361e+09,...,2.200000,91.910000,0.800000,5.000000,5.000000,1.000000,5.321014e+07,28.000000,2.307571e+08,1.000000


In [21]:
train.to_csv("./trainnew.csv", index=False)

In [22]:
new_train = pd.read_csv("./trainnew.csv")

In [23]:
new_train.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,0.0,-999.000000,1.129518,0.044335,1
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,-999.0,0.0,1.0,349.805730,1.620483,0.322436,1
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,-999.0,956940.0,-999.0,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,0.0,-999.000000,1.513370,0.011640,0
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,-999.0,0.0,0.0,89.940100,0.664452,0.082729,1
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,1.0,97.887502,1.427891,0.045630,1


In [24]:
new_train.corr()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
form_field1,1.000000,0.066395,0.358717,0.359764,0.355599,0.155828,0.123642,0.230769,0.229502,0.172198,...,0.413058,0.381166,0.579925,0.240394,0.281385,-0.010492,0.054497,0.361472,0.040527,0.140965
form_field2,0.066395,1.000000,0.080174,0.080579,0.080074,-0.138547,-0.032814,-0.090430,-0.026720,-0.027382,...,0.070366,0.080117,0.094005,-0.021014,0.105363,-0.011653,-0.002065,0.080659,0.024883,-0.073846
form_field3,0.358717,0.080174,1.000000,0.999254,0.993343,0.025861,0.024715,0.038259,0.041953,0.031477,...,0.491087,0.787299,0.241327,0.067352,0.127518,0.001476,0.010087,0.999499,0.006288,-0.009359
form_field4,0.359764,0.080579,0.999254,1.000000,0.991149,0.026420,0.025235,0.039185,0.042613,0.032431,...,0.490614,0.786950,0.242454,0.067842,0.133351,0.001685,0.010339,0.998938,0.006456,-0.009983
form_field5,0.355599,0.080074,0.993343,0.991149,1.000000,0.021725,0.020352,0.032050,0.034483,0.025820,...,0.488447,0.780580,0.232281,0.056543,0.121485,0.001721,0.008272,0.992554,0.005129,-0.015713
form_field6,0.155828,-0.138547,0.025861,0.026420,0.021725,1.000000,0.207660,0.522087,0.289385,0.258334,...,0.018392,0.009114,0.092688,0.179975,-0.005218,0.001275,0.030454,0.025554,0.067747,0.173197
form_field7,0.123642,-0.032814,0.024715,0.025235,0.020352,0.207660,1.000000,0.357153,0.293446,0.900846,...,0.045296,0.038585,0.104369,0.235327,0.053012,-0.005508,0.073881,0.025518,0.063251,0.120380
form_field8,0.230769,-0.090430,0.038259,0.039185,0.032050,0.522087,0.357153,1.000000,0.613040,0.472565,...,0.054251,0.053338,0.162012,0.261659,0.027039,0.009029,0.131430,0.040402,0.134541,0.238216
form_field9,0.229502,-0.026720,0.041953,0.042613,0.034483,0.289385,0.293446,0.613040,1.000000,0.497936,...,0.090142,0.091385,0.164731,0.348559,0.093447,0.006748,0.154381,0.046301,0.224912,0.228493
form_field10,0.172198,-0.027382,0.031477,0.032431,0.025820,0.258334,0.900846,0.472565,0.497936,1.000000,...,0.066230,0.062939,0.136276,0.319098,0.094964,-0.001483,0.112312,0.033265,0.102562,0.169563


In [25]:
train.drop([columns[0]], inplace= True, axis=1)

In [26]:
test.drop([column[0]], inplace= True, axis=1)

In [27]:
train.head()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,...,0.392854,2.02,0.711632,0.0,0.0,0.0,-999.000000,1.129518,0.044335,1
1,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,...,0.314281,8.08,0.183584,-999.0,0.0,1.0,349.805730,1.620483,0.322436,1
2,3276.0,0.53845,3.1510,0.0,6.282,-999.0,956940.0,-999.0,192944.0,1079864.0,...,0.162965,18.18,0.791136,0.0,0.0,0.0,-999.000000,1.513370,0.011640,0
3,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,...,0.488884,2.02,0.685168,-999.0,0.0,0.0,89.940100,0.664452,0.082729,1
4,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,...,0.275000,12.12,0.438168,0.0,0.0,1.0,97.887502,1.427891,0.045630,1


In [28]:
features = train.select_dtypes(exclude = object). columns.drop(['default_status'])

In [29]:
for col in features:
    train[col].fillna(-999, inplace = True)
    test[col].fillna(-999, inplace = True)

In [30]:
X = train[features]
y = train['default_status']

In [31]:
def metric(y, pred):
    return roc_auc_score(y, pred, labels=[0,1])

In [32]:
n_skf = 15
kf = StratifiedKFold(n_skf)

seed = 2020

params = {'n_estimators': 4000, 
          'learning_rate': 0.01, 
          'max_depth': 8, 
          'objective': ' CrossEntropy',
         'eval_metric': 'AUC',
         'random_seed': seed,
         'early_stopping_rounds': 200,
         'use_best_model': True,
}

In [33]:
score_list = []
score = 0
test_oofs = []

for i, (tr_idx, vr_idx) in enumerate(kf.split(X, y)):
    
    xtrain, ytrain = X.loc[tr_idx, features], y.loc[tr_idx]
    xval, yval = X.loc[vr_idx, features], y.loc[vr_idx]
    
    model = CatBoostClassifier(**params)
    model.fit(xtrain, ytrain, eval_set=[(xval,yval)], verbose=100)
    
    p= model.predict_proba(xval)[:, 1]
    sc= metric(yval, p)
    score_list.append(sc)
    score += sc/n_skf
    
    pred = model.predict_proba(test[features])[:, 1]
    test_oofs.append(pred)
    
    print('Fold {} : {}'.format(i, sc))
    
print()
print()
print('Avg log : ', score)

CatBoostError: c:/program files (x86)/go agent/pipelines/buildmaster/catboost.git/catboost/private/libs/options/loss_description.cpp:17:  CrossEntropy loss is not supported